### Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Progetto AN2DL/6_SentModels

Mounted at /gdrive
/gdrive/My Drive/Progetto AN2DL/6_SentModels


### Import libraries

In [2]:
# Hide warnings
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np

import logging

In [3]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print(tf.__version__)

2.14.0


In [4]:
# Import other libraries
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
import pickle
from tqdm import tqdm

### Load and process the dataset

In [5]:
# Load data
data = np.load('../clean_data.npz', allow_pickle=True)

# Save labels in a new list (image pixel values are float in [0, 1])
images = data['data']

# Save labels in a new list
labels = data['labels']

# Print data shape
print(images.shape, labels.shape)

(5004, 96, 96, 3) (5004,)


In [6]:
# Convert labels from string to integer
num_labels = []
for label in labels:
  if label == "healthy":
    num_labels.append(0)
  else:
    num_labels.append(1)

# Convert labels in the one-hot encoding format
num_labels = np.array(num_labels)
labels = tfk.utils.to_categorical(num_labels, 2)

### ConvNeXtLarge

In [7]:
# Import Feature Extractor with specified settings
feature_extractor = tfk.applications.ConvNeXtLarge(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
    include_preprocessing=True,
)

785596384/785596384 [==============================] - 9s 0us/step


### Initial setup

In [8]:
# Define network parameters/callbacks
dropout_rate = 1/10
learning_rate = 1e-3

lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=5,
    factor=0.99,
    mode='max',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=15,
    restore_best_weights=True
)

In [9]:
def build_model(feature_extractor, dropout_rate, learning_rate=0.001):
  # Use the supernet as feature extractor, i.e. freeze all its weigths
  feature_extractor.trainable = False

  # Create an input layer with shape (96, 96, 3)
  inputs = tfk.Input(shape=(96, 96, 3), name='Input')

  # Connect FeatureExtractor to the input
  x = feature_extractor(inputs)

  # Hidden layers
  x = tfkl.Dense(units=256, name='HiddenDense1')(x)
  x = tfkl.Activation('relu', name='HiddenActivation1')(x)
  dropout = tfkl.Dropout(dropout_rate)(x)
  x = tfkl.Dense(units=256, name='HiddenDense2')(dropout)
  x = tfkl.Activation('relu', name='HiddenActivation2')(x)
  dropout = tfkl.Dropout(dropout_rate)(x)

  # Add a Dense layer with 2 unit and softmax activation as the classifier
  outputs = tfkl.Dense(2, activation='softmax')(dropout)

  # Create a Model connecting input and output
  model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

  # Compile the model with Categorical Cross-Entropy loss and Adam optimizer
  model.compile(loss=tfk.losses.CategoricalCrossentropy(),
                optimizer=tfk.optimizers.Adam(learning_rate),
                metrics=['accuracy'])

  # Return the model
  return model

### K-Fold + Transfer Learning

In [11]:
# Define the number of folds for cross-validation
num_folds = 10

# Initialize lists to store training histories, scores, and best epochs
histories = []
scores = []
best_epochs = []

# Create a KFold cross-validation object
kfold = KFold(n_splits=num_folds, shuffle=True)

# Loop through each fold
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(images, labels)):

  print("Starting training on fold num: {}".format(fold_idx+1))

  # Init model
  k_model = build_model(feature_extractor, dropout_rate, learning_rate)
  train_datagen = ImageDataGenerator(
          horizontal_flip = True,
          vertical_flip = True,
          width_shift_range = 0.1,
          height_shift_range = 0.1,
          rotation_range=10.,
          fill_mode='reflect')

  train_datagen.fit((images)[train_idx]*255)

  # Train the model on the training data for this fold
  history = k_model.fit_generator(
      train_datagen.flow((images)[train_idx]*255, labels[train_idx], batch_size=16),
      epochs=200,
      steps_per_epoch=len((images)[train_idx])/16,
      validation_data = ((images)[valid_idx]*255, labels[valid_idx]),
      callbacks = [early_stopping, lr_scheduler]
      ).history

  # Evaluate the model on the validation data for this fold
  score = k_model.evaluate((images)[valid_idx]*255, labels[valid_idx], verbose=0)
  scores.append(score[1])

  # Calculate the best epoch for early stopping
  best_epoch = len(history['loss']) - 15
  best_epochs.append(best_epoch)

  # Store the training history for this fold
  histories.append(history)

Starting training on fold num: 1
Epoch 1/200
281/281 [==============================] - 80s 174ms/step - loss: 0.5180 - accuracy: 0.7517 - val_loss: 0.3755 - val_accuracy: 0.8443 - lr: 0.0010
Epoch 2/200
281/281 [==============================] - 30s 107ms/step - loss: 0.3892 - accuracy: 0.8306 - val_loss: 0.3844 - val_accuracy: 0.8343 - lr: 0.0010
Epoch 3/200
281/281 [==============================] - 29s 103ms/step - loss: 0.3322 - accuracy: 0.8539 - val_loss: 0.3387 - val_accuracy: 0.8443 - lr: 0.0010
Epoch 4/200
281/281 [==============================] - 31s 108ms/step - loss: 0.3148 - accuracy: 0.8621 - val_loss: 0.3410 - val_accuracy: 0.8563 - lr: 0.0010
Epoch 5/200
281/281 [==============================] - 31s 112ms/step - loss: 0.2910 - accuracy: 0.8761 - val_loss: 0.3012 - val_accuracy: 0.8583 - lr: 0.0010
Epoch 6/200
281/281 [==============================] - 30s 105ms/step - loss: 0.2832 - accuracy: 0.8803 - val_loss: 0.2882 - val_accuracy: 0.8743 - lr: 0.0010
Epoch 7/200
2

In [ ]:
# Define a list of colors for plotting
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Create a figure for Validation visualization
for fold_idx in range(num_folds):
  plt.plot(histories[fold_idx]['val_accuracy'][:-15], color=colors[fold_idx], label=f'Fold N°{fold_idx+1}')
  plt.ylim(0.7, 1.0)
  plt.title('Accuracy')
  plt.legend(loc='lower left')
  plt.grid(alpha=.3)

# Show the plot
plt.show()

In [13]:
# Calculate the average best epoch
print (best_epochs)
avg_epochs = int(np.mean(best_epochs))
print(f"Best average epoch: {avg_epochs}")

[25, 39, 32, 14, 20, 15, 13, 8, 16, 19]
Best average epoch: 20


In [10]:
avg_epochs = 20

# Build the final model using the calculated average best epoch
final_model = build_model(feature_extractor, dropout_rate, learning_rate)
train_datagen = ImageDataGenerator(
        horizontal_flip = True,
        vertical_flip = True,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        rotation_range=10.,
        fill_mode='reflect')

train_datagen.fit(images*255)

# Train the final model on the combined training and validation data
final_history = final_model.fit_generator(
      train_datagen.flow((images)*255, labels, batch_size=16),
      epochs=avg_epochs,
      steps_per_epoch=len(images)/16,
      validation_data = ((images)*255, labels),
      callbacks = [early_stopping, lr_scheduler]
      ).history

Epoch 1/20
312/312 [==============================] - 115s 271ms/step - loss: 0.5051 - accuracy: 0.7592 - val_loss: 0.3720 - val_accuracy: 0.8243 - lr: 0.0010
Epoch 2/20
312/312 [==============================] - 50s 160ms/step - loss: 0.3875 - accuracy: 0.8317 - val_loss: 0.3527 - val_accuracy: 0.8445 - lr: 0.0010
Epoch 3/20
312/312 [==============================] - 46s 146ms/step - loss: 0.3243 - accuracy: 0.8565 - val_loss: 0.2735 - val_accuracy: 0.8903 - lr: 0.0010
Epoch 4/20
312/312 [==============================] - 48s 153ms/step - loss: 0.3165 - accuracy: 0.8633 - val_loss: 0.2969 - val_accuracy: 0.8747 - lr: 0.0010
Epoch 5/20
312/312 [==============================] - 54s 172ms/step - loss: 0.2909 - accuracy: 0.8735 - val_loss: 0.2868 - val_accuracy: 0.8819 - lr: 0.0010
Epoch 6/20
312/312 [==============================] - 42s 135ms/step - loss: 0.2815 - accuracy: 0.8801 - val_loss: 0.2728 - val_accuracy: 0.8769 - lr: 0.0010
Epoch 7/20
312/312 [==============================]

In [11]:
# Save the model
final_model.save('2ndSubmission_TL')
with open('2ndSubmission_TL/history.pkl', 'wb') as f:
  pickle.dump(final_history, f)

### Fine Tuning

In [ ]:
# Define the number of folds for cross-validation
num_folds = 10

# Initialize lists to store training histories, scores, and best epochs
histories = []
scores = []
best_epochs = []

# Create a KFold cross-validation object
kfold = KFold(n_splits=num_folds, shuffle=True)

# Loop through each fold
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(images, labels)):

  print("Starting training on fold num: {}".format(fold_idx+1))

  # Init model
  k_model = tfk.models.load_model('2ndSubmission_TL')
  train_datagen = ImageDataGenerator(
          horizontal_flip = True,
          vertical_flip = True,
          width_shift_range = 0.1,
          height_shift_range = 0.1,
          rotation_range=10.,
          fill_mode='reflect')

  train_datagen.fit((images)[train_idx]*255)

  # Set all ConvNeXtLarge layers as trainable
  k_model.get_layer('convnext_large').trainable = True
  for i, layer in enumerate(k_model.get_layer('convnext_large').layers):
    print(i, layer.name, layer.trainable)

  # Freeze first N layers, e.g., until the 133rd one
  N = 286
  for i, layer in enumerate(k_model.get_layer('convnext_large').layers[:N]):
    layer.trainable=False
  for i, layer in enumerate(k_model.get_layer('convnext_large').layers):
    print(i, layer.name, layer.trainable)
  k_model.summary()

  # Compile the model
  k_model.compile(loss=tfk.losses.CategoricalCrossentropy(),
                  optimizer=tfk.optimizers.Adam(learning_rate),
                  metrics='accuracy')

  # Train the model on the training data for this fold
  history = k_model.fit_generator(
      train_datagen.flow((images)[train_idx]*255, labels[train_idx], batch_size=16),
      epochs=200,
      steps_per_epoch=len((images)[train_idx])/16,
      validation_data = ((images)[valid_idx]*255, labels[valid_idx]),
      callbacks = [early_stopping, lr_scheduler]
      ).history

  # Evaluate the model on the validation data for this fold
  score = k_model.evaluate((images)[valid_idx]*255, labels[valid_idx], verbose=0)
  scores.append(score[1])

  # Calculate the best epoch for early stopping
  best_epoch = len(history['loss']) - 15
  best_epochs.append(best_epoch)

  # Store the training history for this fold
  histories.append(history)

In [ ]:
# Define a list of colors for plotting
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Create a figure for Validation visualization
for fold_idx in range(num_folds):
  plt.plot(histories[fold_idx]['val_accuracy'][:-15], color=colors[fold_idx], label=f'Fold N°{fold_idx+1}')
  plt.ylim(0.7, 1.0)
  plt.title('Accuracy')
  plt.legend(loc='lower left')
  plt.grid(alpha=.3)

# Show the plot
plt.show()

In [ ]:
# Calculate the average best epoch
print (best_epochs)
avg_epochs = int(np.mean(best_epochs))
print(f"Best average epoch: {avg_epochs}")

In [ ]:
# Build the final model using the calculated average best epoch
final_model = tfk.models.load_model('2ndSubmission_TL')
train_datagen = ImageDataGenerator(
        horizontal_flip = True,
        vertical_flip = True,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        rotation_range=10.,
        fill_mode='reflect')

train_datagen.fit((images)*255)

# Set all ConvNeXtLarge layers as trainable
final_model.get_layer('convnext_large').trainable = True
for i, layer in enumerate(final_model.get_layer('convnext_large').layers):
  print(i, layer.name, layer.trainable)

# Freeze first N layers, e.g., until the 133rd one
N = 286
for i, layer in enumerate(final_model.get_layer('convnext_large').layers[:N]):
  layer.trainable=False
for i, layer in enumerate(final_model.get_layer('convnext_large').layers):
  print(i, layer.name, layer.trainable)
final_model.summary()

# Compile the model
final_model.compile(loss=tfk.losses.CategoricalCrossentropy(),
                optimizer=tfk.optimizers.Adam(learning_rate),
                metrics='accuracy')

# Train the final model on the combined training and validation data
final_history = final_model.fit_generator(
      train_datagen.flow((images)*255, labels, batch_size=16),
      epochs=avg_epochs,
      steps_per_epoch=len(images)/16,
      validation_data = ((images)*255, labels),
      callbacks = [early_stopping, lr_scheduler]
      ).history

In [ ]:
# Save the model
final_model.save('2ndSubmission_FT')
with open('2ndSubmission_FT/history.pkl', 'wb') as f:
  pickle.dump(final_history, f)